In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
import re
import nltk
from nltk.corpus import stopwords
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
df=pd.read_csv("/kaggle/input/samsum-dataset-text-summarization/samsum-train.csv")
df=df.drop(columns=["id"])

In [3]:
df.dropna(how="any",axis=0)

,dialogue,summary
0,Amanda: I baked cookies. Do you want some?\r\...,Amanda baked cookies and will bring Jerry some...
1,Olivia: Who are you voting for in this electio...,Olivia and Olivier are voting for liberals in ...
2,"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa...",Kim may try the pomodoro technique recommended...
3,"Edward: Rachel, I think I'm in ove with Bella....",Edward thinks he is in love with Bella. Rachel...
4,Sam: hey overheard rick say something\r\nSam:...,"Sam is confused, because he overheard Rick com..."
...,...,...
14727,Romeo: You are on my ‘People you may know’ lis...,Romeo is trying to get Greta to add him to her...
14728,Theresa: <file_photo>\r\nTheresa: <file_photo>...,Theresa is at work. She gets free food and fre...
14729,John: Every day some bad news. Japan will hunt...,Japan is going to hunt whales again. Island an...
14730,Jennifer: Dear Celia! How are you doing?\r\nJe...,Celia couldn't make it to the afternoon with t...


In [4]:
df.dtypes

dialogue    object
summary     object
dtype: object

In [5]:
def text_preprocess(text):
    
    data=text.lower() if isinstance(text,str) else "nil"
    data_re=re.sub(r"\W+"," ",data)
    data_word_tokenize=word_tokenize(data_re)
    tokens=[word for word in data_word_tokenize if word not in stopwords.words("english")]
    return " ".join(tokens)
def text_vectorizer(text):
    vectorizer=TfidfVectorizer()
    data=text.apply(text_preprocess)
    vect=vectorizer.fit_transform(data)
    ax=pd.DataFrame(vect.toarray(),columns=vectorizer.get_feature_names_out())
    return ax

In [6]:
d=text_vectorizer(df["dialogue"])
X=text_vectorizer(df["summary"])

In [7]:
concaty=pd.concat([d,X],axis=0)

In [8]:
con=concaty.dropna(how='any',axis=1)
con.isnull().sum().sum()

0

In [9]:
con.shape

(29464, 14924)

In [10]:
!pip install cirq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 31.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.7/532.7 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.3/69.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.5/596.5 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.8/202.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 78.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 54.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 66.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 50.1 MB/s e

In [11]:
from keras.layers import Layer,Dense,Flatten,BatchNormalization,Input,LSTM,SimpleRNN,GRU,LeakyReLU
from keras.models import Sequential,Model
from keras.initializers import HeNormal
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
import cirq
import tensorflow as tf

In [12]:
inputs=Input(shape=(14924,))
D=Dense(32,activation="relu")(inputs)
D=Dense(64,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
D=LeakyReLU(0.08)(D)
D=Dense(128,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
D=LeakyReLU(0.08)(D)
D=Dense(256,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
D=LeakyReLU(0.08)(D)

D=Dense(556,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
D=LeakyReLU(0.08)(D)

D=Dense(256,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
D=LeakyReLU(0.08)(D)
D=Dense(128,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
D=LeakyReLU(0.08)(D)
D=Dense(64,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
D=LeakyReLU(0.08)(D)
D=Dense(32,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
D=LeakyReLU(0.08)(D)
outputs=Dense(1)(D)
model=Model(inputs,outputs)
optimizer=Adam(learning_rate=0.01,beta_1=0.99,beta_2=0.99)
model.compile(optimizer='adam',loss="mse",metrics=["mae"])
model.fit(con,con,epochs=5,validation_split=0.2)


Epoch 1/5
737/737 ━━━━━━━━━━━━━━━━━━━━ 18s 19ms/step - loss: 7.4415 - mae: 7.2109e-04 - val_loss: 0.0058 - val_mae: 4.2223e-04
Epoch 2/5
737/737 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - loss: 0.0021 - mae: 4.4027e-04 - val_loss: 6.7426e-05 - val_mae: 4.3654e-04
Epoch 3/5
737/737 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - loss: 6.4529e-05 - mae: 4.4157e-04 - val_loss: 6.6197e-05 - val_mae: 4.0166e-04
Epoch 4/5
737/737 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - loss: 6.4128e-05 - mae: 4.4248e-04 - val_loss: 6.6224e-05 - val_mae: 4.6340e-04
Epoch 5/5
737/737 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - loss: 6.4112e-05 - mae: 4.4097e-04 - val_loss: 6.6235e-05 - val_mae: 3.1901e-04


In [13]:
predict=model.predict(con.iloc[:50,:])

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step


In [14]:
predict

array([[0.00012329],
       [0.00012329],
       [0.00012329],
       [0.00012329],
       [0.00012329],
       [0.00012329],
       [0.00012329],
       [0.00012329],
       [0.00012329],
       [0.00012329],
       [0.00012329],
       [0.00012329],
       [0.00012329],
       [0.00012329],
       [0.00012329],
       [0.00012329],
       [0.00012329],
       [0.00012329],
       [0.00012329],
       [0.00012329],
       [0.00012329],
       [0.00012329],
       [0.00012329],
       [0.00012329],
       [0.00012329],
       [0.00012329],
       [0.00012329],
       [0.00012329],
       [0.00012329],
       [0.00012329],
       [0.00012329],
       [0.00012329],
       [0.00012329],
       [0.00012329],
       [0.00012329],
       [0.00012329],
       [0.00012329],
       [0.00012329],
       [0.00012329],
       [0.00012329],
       [0.00012329],
       [0.00012329],
       [0.00012329],
       [0.00012329],
       [0.00012329],
       [0.00012329],
       [0.00012329],
       [0.000

# # GAN

In [80]:
def build_gen():
    inputs=Input(shape=(100,))
    g=Dense(256,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(inputs)
    g=LeakyReLU(0.08)(g)
    g=Dense(128,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(g)
    g=LeakyReLU(0.08)(g)
    g=Dense(64,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(g)
    g=LeakyReLU(0.08)(g)
    g=Dense(32,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(g)
    g=LeakyReLU(0.08)(g)
    outputs=Dense(14924)(g)
    model=Model(inputs,outputs)
    return model
def build_dis():
    inputs=Input(shape=(14924,))
    g=Dense(32,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(inputs)
    g=LeakyReLU(0.08)(g)
    g=Dense(64,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(g)
    g=LeakyReLU(0.08)(g)
    g=Dense(128,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(g)
    g=LeakyReLU(0.08)(g)
    g=Dense(256,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(g)
    g=LeakyReLU(0.08)(g)
    outputs=Dense(1)(g)
    model=Model(inputs,outputs)
    return model
def train_gan(gen,dis,batch_size):
    dis.trainable=False
    noise=Input(shape=(100,))
    fake=gen(noise)
    outputs=dis(fake)
    model=Model(noise,outputs)
    return model
def build_model(gen,dis,gan,data,batch_size,epochs):
    dis.trainable=True
#     gan.trainable=True
    dis.compile(optimizer="adam",loss="mse",metrics=["mae"])
    gan.compile(optimizer="adam",loss="mse",metrics=["mae"])
    for epoch in range(epochs):
        for batch_data in data:
            if isinstance(batch_data, str):
                batch_dat = text_vectorizer(batch_data)
            real_data=batch_dat[:batch_size]

#             if not np.issubdtype(real_data.dtype, np.number):
#                 real_data = np.where(np.vectorize(lambda x: isinstance(x, (int, float)))(real_data), real_data, 0)
            real_label=np.zeros((batch_size,1))
            fake_label=np.ones((batch_size,1))
            noise=np.random.normal(0,1,(batch_size,100))
            fake_data=gen.predict(noise)
            dis.trainable=True
            dis.train_on_batch(real_data,real_label)
            dis.train_on_batch(fake_data,fake_label)
            dis.trainable=False
            gan.train_on_batch(noise,real_label)
#             dis.trainable=True
    return gan
gen=build_gen()
dis=build_dis()
gan=train_gan(gen,dis,32)
build_model(gen,dis,gan,con,32,5)

AttributeError: 'str' object has no attribute 'apply'

In [81]:
con.isnull().sum().sum()

0

In [82]:
real_data=con.iloc[:32,:]
real_data

,00,000,01,01753,03,0333,03843,047839898,05,07,...,zoya,zoé,zubac,zuccarini,zulczyk,zuleyka,zumba,zuri,zuza,élysées
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [83]:
# con.dtypes.sum()

In [84]:
# from transformers import LlamaForCausalLM, LlamaTokenizer


In [85]:
# tokenize=LlamaTokenizer.from_pretrained("src/transformers/models/llama/modeling_llama.py")
# model=LlamaForCasualLM.from_pretrained("src/transformers/models/llama/modeling_llama.py")
# inputs=tokenize(df)
# encode=model.generate(inputs,max_length)
# decode=tokenize.decode(encode)
# print(decode)

In [86]:
import torch
import torch.nn as nn
class transformer_model(data):
    def __init__(self):
    super(transformer_model,self).__init__()
    self.attention=MultiHeadAttention()
    self.layer=nn.LayerNorm()
#     self.linear=nn.LayerNorm()
    self.embedding=nn.Embedding
    self.module=ModuleLIst()
    self.dropout=nn.Dropout()
    self.feed_forward=nn.Sequential(
            nn.Linear(),
            nn.ReLU(),
            nn.Linear())
def feed_forward():
    attention=self.attention()
    x=self.dropout(self.layer())
    forward=self.feed_forward(attention+x)
    out=self.dropout(forward+x)
    return output
class simple_feature(data):
    def __init__(self):
        super(simple_feature,self).__init__()
        self.word_embedding=nn.Embedding()
        self.linear=nn.Linear()
        self.dropout=nn.Dropout()

IndentationError: expected an indented block after function definition on line 4 (1261892914.py, line 5)